In [1]:
# Context Manager that goes in and stores variables to a dictionary at the highest level of the NASEM_execute() function

class Test_DictContext:
    def __init__(self, results_dict):
        self.results_dict = results_dict
    
    def __enter__(self):
        self.original_locals = dict(locals())
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        # Get the calling frame
        calling_frame = inspect.currentframe().f_back
        # Get the local variables of the calling frame
        current_locals = calling_frame.f_locals
        
        for key, value in current_locals.items():
            # Flag to check if the variable was categorized into any group
            categorized = False
            for group_key, group_value in result_dict.items():
                if key in group_value:
                    result_dict[group_key][key] = current_locals[key]
                    categorized = True # Flag all variables that get stored
                    break  
            if not categorized:
                # Store any variables that aren't included in a group
                result_dict['Uncategorized'][key] = current_locals[key]     



# Rewrite this context manager as a function
def variable_storage(result_dict):
    # Get the calling frame
    calling_frame = inspect.currentframe().f_back
    # Get the local variables of the calling frame
    current_locals = calling_frame.f_locals
        
    for key, value in current_locals.items():
        # Flag to check if the variable was categorized into any group
        categorized = False
        for group_key, group_value in result_dict.items():
            if key in group_value:
                result_dict[group_key][key] = current_locals[key]
                categorized = True # Flag all variables that get stored
                break  
        if not categorized:
            # Store any variables that aren't included in a group
            result_dict['Uncategorized'][key] = current_locals[key] 

    return result_dict  


                              


In [2]:
# Modify function to ignore result_dict
import inspect

def variable_storage_v2(result_dict):
    # Get the calling frame
    calling_frame = inspect.currentframe().f_back
    # Get the local variables of the calling frame
    current_locals = calling_frame.f_locals
        
    for key, value in current_locals.items():
        # Flag to check if the variable was categorized into any group
        categorized = False
        for group_key, group_value in result_dict.items():
            if key in group_value:
                result_dict[group_key][key] = current_locals[key]
                categorized = True # Flag all variables that get stored
                break  
        if not categorized:
            # Catch result_dict and prevent storing inside self; can also catch other variables used to pass data to the functions
            if key == 'result_dict' or key == 'input_values':
                pass
            else:
                # Store any variables that aren't included in a group
                result_dict['Uncategorized'][key] = current_locals[key] 

    return result_dict 



In [3]:
# Define functions that act like the calculation functions in the model

def calculate_value1(input_values, result_dict, NASEM_model=False):
    variable_1_1 = input_values['num_1'] * 10
    variable_1_2 = input_values['num_2'] * input_values['num_3']
    variable_1_3 = variable_1_1 + variable_1_2

    if NASEM_model is True:
        variable_storage_v2(result_dict)

    return variable_1_3


def calculate_value2(input_values, result_dict, NASEM_model=False):
    variable_2_1 = 50 + input_values['num_4']
    variable_2_2 = variable_2_1 / 5 - input_values['num_1']

    if NASEM_model is True:
        variable_storage_v2(result_dict)

    return variable_2_2



def calculate_value3(input_values, result_dict, NASEM_model=False):
    variable_3_1 = input_values['num_1'] + input_values['num_5']
    variable_3_2 = 10 - variable_3_1

    if NASEM_model is True:
        variable_storage_v2(result_dict)

    return variable_3_2

    

In [25]:
# Creating a sample function with same structure as NASEM_model()
# Will pass value to fucntions from a dict in test_model, use to perform calculations
# all variables inside the functions will then be stored in results_dict inside test_model

def test_model():
# Replicate animal_input, diet_info, etc.
# These values are passed to calculation functions
    input_values = {
        'num_1': 50,
        'num_2': 10,
        'num_3': 2.459,
        'num_4': 5.28,
        'num_5': 1.5
    }

# This is where all the model results will be sent 
    result_dict = {
        'Group 1': {
            'variable_1_1': None,
            'variable_1_2': None,
            'variable_1_3': None
        },
        'Group 2': {
            'variable_2_1': None,
            'variable_2_2': None
        },
        'Group 3': {
            'variable_3_1': None,
            'variable_3_2': None
        },
        'Uncategorized': {}
    }
    print(result_dict)

    variable_1_3 = calculate_value1(input_values, result_dict, NASEM_model=True)

    variable_2_2 = calculate_value2(input_values, result_dict, NASEM_model=True)

    variable_3_2 = calculate_value3(input_values, result_dict, NASEM_model=True)

    print('\n\n')
    print(result_dict)
    return result_dict


##### RUN MODEL #####
result_dict = test_model()


{'Group 1': {'variable_1_1': None, 'variable_1_2': None, 'variable_1_3': None}, 'Group 2': {'variable_2_1': None, 'variable_2_2': None}, 'Group 3': {'variable_3_1': None, 'variable_3_2': None}, 'Uncategorized': {}}



{'Group 1': {'variable_1_1': 500, 'variable_1_2': 24.59, 'variable_1_3': 524.59}, 'Group 2': {'variable_2_1': 55.28, 'variable_2_2': -38.944}, 'Group 3': {'variable_3_1': 51.5, 'variable_3_2': -41.5}, 'Uncategorized': {'NASEM_model': True}}


In [4]:
# Show how fucntions can be run outside model
input_numbers = {
    'num_1': 5,
    'num_2': 12,
    'num_3': -3
}

# Need a dictionary because of how I setup these demo functions
dict = {}

# By default savung to a dictionary can be turned off or on
variable_1_3 = calculate_value1(input_numbers, dict)



In [8]:
# Can be used to extract values from functions outside of model

# User defines a dictionary including all the variables they want out of there functions
a_nested_dictionary = {
    'Group 1': {
        'variable_1_2': None
    },
    'Uncategorized': {}
}

input_numbers = {
    'num_1': 5,
    'num_2': 12,
    'num_3': -3
}

# User calls a package fucntion 
variable_1_3 = calculate_value1(input_numbers, a_nested_dictionary, NASEM_model=True)

# Normally only variable_1_3 would be returned from this function. By using this dictionary method users can extract all 
# the variables they want from any of these functions by defining a dictionary
print(a_nested_dictionary)


{'Group 1': {'variable_1_2': -36}, 'Uncategorized': {'NASEM_model': True, 'variable_1_1': 50, 'variable_1_3': 14}}
